Status: complete

Data
- Benchling notebook: [2024-11-07 Human-mouse mixing ChIP-seq](https://benchling.com/s/etr-GcNccQeNTC32BHI3mLna)
- [Library Sheet](https://docs.google.com/spreadsheets/d/1TwqZjgeley2BaOh1xkQ5Pwzi_yxNvlc1KeaTBwyCLiU)
- Sequencing runs:
  - [20241121_ESS](https://docs.google.com/spreadsheets/d/1kV8gOaSSB9VktkTEqVO4W3izpBkpOZ6Du1kDDAOLh8c)

Splitcode configs: https://docs.google.com/spreadsheets/d/1XrNNYC3ckBm_Ir2E3MWDkFMDMuHjL5Z7uOmD9kNKZOE

Results

Conclusions

# Setup

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext watermark

In [ ]:
# Python Standard Library modules
import collections
import copy
import gc
import gzip
import importlib.util
import itertools
import json
import os
import re
import sys

# Basic SciPy packages
import numpy as np
import scipy
import pandas as pd

# Visualization packages
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.auto import tqdm
import IPython.display

import pysam

# ChIP-DIP modules
path_chipdip_scripts = '/central/groups/guttman/projects/chip-dip/github_private/scripts/python'
module_spec = importlib.util.spec_from_file_location('rename_and_filter_chr', os.path.join(path_chipdip_scripts, 'rename_and_filter_chr.py'))
rename_and_filter_chr = importlib.util.module_from_spec(module_spec)
module_spec.loader.exec_module(rename_and_filter_chr)

# Custom modules
sys.path.append('../scripts')
import string_distances
import parse_barcodes
from helpers import fastq_parse
sys.path.remove('../scripts')

In [ ]:
%watermark --updated --iso8601 --python --conda --machine --iversions --watermark

In [ ]:
%%bash
splitcode --version

In [ ]:
DIR_PROJECT = '/central/groups/guttman/btyeh/scBarcode'
DIR_DATA = os.path.join(DIR_PROJECT, 'data', '20241121')
DIR_AUX = os.path.join(DIR_PROJECT, 'data_aux', '20241121')
DIR_PROC = os.path.join(DIR_PROJECT, 'data_proc', '20241121')
DIR_RESULTS = os.path.join(DIR_PROJECT, 'results', '20241121')
DIR_SCRIPTS = os.path.join(DIR_PROJECT, 'scripts', '20241121')

os.makedirs(DIR_AUX, exist_ok=True)
os.makedirs(DIR_PROC, exist_ok=True)
os.makedirs(DIR_RESULTS, exist_ok=True)

In [ ]:
def estimate_library_complexity(count_total, count_dedup, ub=None, max_err=1e-3):
    '''
    See https://github.com/bentyeh/resources/blob/main/bioinformatics/models_genomics.md

    Generative model: Poisson sampling (i.e., with replacement) count_total reads from
    M unique molecules, such that count_dedup molecules were sampled at least once. Solve
    for M.

    Accurate when count_total << M.
    '''
    if ub is None:
        ub = count_total*1e5
    res = scipy.optimize.minimize_scalar(
      fun=lambda M: (M * (1 - np.exp(-count_total/M)) - count_dedup)**2,
      bracket=(count_dedup, ub)
    )
    assert res.fun < max_err
    return res.x

def estimate_library_complexity2(count_total, count_mean, ub=None, max_err=1e-3):
    '''
    See https://github.com/bentyeh/resources/blob/main/bioinformatics/models_genomics.md

    Generative model: Poisson sampling (i.e., with replacement) count_total reads from
    M unique molecules, yielding mean observed counts count_mean. Solve for M.
    '''
    ub = ub if ub is not None else count_total*1e5
    res = scipy.optimize.minimize_scalar(
      fun=lambda M: ((count_total / M) / (1 - np.exp(-count_total / M)) - count_mean)**2,
      bracket=(count_total / count_mean, ub)
    )
    assert res.fun < max_err
    return res.x

def estimate_library_complexity_curve(
    total_reads: np.ndarray,
    M: int,
    ci: tuple[float, float] = (0.025, 0.975)
) -> pd.DataFrame:
    '''
    Args
    - total_reads
    - M: total number of unique molecules in the sample
    - ci: confidence interval

    Returns: pd.DataFrame
    - Columns = total_reads, expected_distinct, lower_ci, upper_ci
    '''
    p_zero = scipy.stats.binom.pmf(0, total_reads, 1/M)
    expected_distinct = M * (1 - p_zero)

    # smallest d such that P(D < d) > ci[0]
    lower_ci = scipy.stats.binom.ppf(ci[0], M, 1 - p_zero)

    # smallest d such that P(D < d) > ci[1]
    upper_ci = scipy.stats.binom.ppf(ci[1], M, 1 - p_zero)
    return pd.DataFrame({
        'total_reads': total_reads,
        'expected_distinct': expected_distinct,
        'lower_ci': lower_ci,
        'upper_ci': upper_ci
    })

In [ ]:
# from https://docs.python.org/3/library/itertools.html
def grouper(iterable, n, *, incomplete='fill', fillvalue=None):
    "Collect data into non-overlapping fixed-length chunks or blocks."
    # grouper('ABCDEFG', 3, fillvalue='x') → ABC DEF Gxx
    # grouper('ABCDEFG', 3, incomplete='strict') → ABC DEF ValueError
    # grouper('ABCDEFG', 3, incomplete='ignore') → ABC DEF
    iterators = [iter(iterable)] * n
    match incomplete:
        case 'fill':
            return itertools.zip_longest(*iterators, fillvalue=fillvalue)
        case 'strict':
            return zip(*iterators, strict=True)
        case 'ignore':
            return zip(*iterators)
        case _:
            raise ValueError('Expected fill, strict, or ignore')

In [ ]:
# number of processes/threads to use
# either manually specify an integer, or automatically detect via something like os.cpu_count()
n_proc = 4

reprocess = False

## Constants

In [ ]:
ROUNDS = ['Odd', 'Even', 'Odd2', 'Even2', 'Odd3', 'Y']

TARGETS = ['CTCF', 'H3K4me3']
DTYPE_TARGET = pd.CategoricalDtype(categories=TARGETS)

SPECIES = ['human', 'mouse']
DTYPE_SPECIES = pd.CategoricalDtype(categories=SPECIES)
DTYPE_SPECIES_ABBREV = pd.CategoricalDtype(categories=['h', 'm'])

ALIGNMENT_TYPES = ['R1', 'PE']
DTYPE_ALIGNMENT = pd.CategoricalDtype(categories=ALIGNMENT_TYPES)

Key paths

In [ ]:
path_bead_counts = os.path.join(DIR_PROC, 'bead_counts.npz')

## Download human and mouse genome indices

Bowtie 2 human genome index

In [ ]:
%%bash
PATH_SCRATCH='/central/scratch/btyeh'

cd "$PATH_SCRATCH"

if [ ! -f "${PATH_SCRATCH}/index_hg38/GRCh38_noalt_as.1.bt2" ] || [ ! -f "${PATH_SCRATCH}/index_hg38/GRCh38_noalt_as.rev.2.bt2" ]; then
    mkdir -p index_hg38
    wget -q https://genome-idx.s3.amazonaws.com/bt/GRCh38_noalt_as.zip
    unzip -j -d index_hg38 GRCh38_noalt_as.zip \*.bt2
fi

Bowtie 2 mouse genome index

In [ ]:
%%bash
PATH_SCRATCH='/central/scratch/btyeh'

cd "$PATH_SCRATCH"

if [ ! -f "${PATH_SCRATCH}/index_mm10/mm10.1.bt2" ] || [ ! -f "${PATH_SCRATCH}/index_mm10/mm10.rev.2.bt2" ]; then
    mkdir -p index_mm10
    wget -q https://genome-idx.s3.amazonaws.com/bt/mm10.zip
    unzip -j -d index_mm10 mm10.zip \*.bt2
fi

## Build combined human-mouse genome index

Download all FASTA files (1 file per chromosome) for human and mouse genomes. Rename human chromosomes from "chr*" to "h_chr*" and mouse chromosomes from "chr*" to "m_chr*"

In [ ]:
%%bash
PATH_SCRATCH='/central/scratch/btyeh/hg38'
PATH_INDEX='/central/scratch/btyeh/index_hg38_mm10'
if [ ! -f "$PATH_SCRATCH"/h_chr1.fa ] && \
    ([ ! -f "$PATH_INDEX"/hg38_mm10.rev.1.bt2l ] || [ ! -f "$PATH_SCRATCH"/h_chr1.fa.fai ]); then
    mkdir -p "$PATH_SCRATCH"
    wget -nc -q -O - https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.chromFa.tar.gz |
        tar -xz -C "$PATH_SCRATCH"
    mv "$PATH_SCRATCH"/chroms/*.fa "$PATH_SCRATCH"
    
    # remove alternate loci sequences
    rm -r "$PATH_SCRATCH"/chroms "$PATH_SCRATCH"/*_alt.fa
    
    cd "$PATH_SCRATCH"
    for file in chr*.fa; do
        new_file="h_${file}"
        sed -e 's/^>chr/>h_chr/' "$file" > "$new_file"
        rm "$file"
    done
fi

In [ ]:
%%bash
PATH_SCRATCH='/central/scratch/btyeh/mm10'
PATH_INDEX='/central/scratch/btyeh/index_hg38_mm10'
if [ ! -f "$PATH_SCRATCH"/m_chr1.fa ] && \
    ([ ! -f "$PATH_INDEX"/hg38_mm10.rev.1.bt2l ] || [ ! -f "$PATH_SCRATCH"/m_chr1.fa.fai ]); then
    mkdir -p "$PATH_SCRATCH"
    wget -nc -q -O - https://hgdownload.soe.ucsc.edu/goldenPath/mm10/bigZips/chromFa.tar.gz |
        tar -xz -C "$PATH_SCRATCH"
    
    cd "$PATH_SCRATCH"
    for file in chr*.fa; do
        new_file="m_${file}"
        sed -e 's/^>chr/>m_chr/' "$file" > "$new_file"
        rm "$file"
    done
fi

Build combined genome index

In [ ]:
%%bash -s {DIR_PROJECT}
DIR_PROJECT="$1"

PATH_SBATCH="${DIR_PROJECT}/scripts/20240625/build_index.sbatch"
PATH_INDEX='/central/scratch/btyeh/index_hg38_mm10'

if [ ! -f "$PATH_INDEX"/hg38_mm10.rev.1.bt2l ]; then
    sbatch --output="$PATH_INDEX"/slurm.out --error="$PATH_INDEX"/slurm.err "$PATH_SBATCH"
fi

### Create IGV Genome JSON file

Concatenate all reference chromosome FASTA files and build a FASTA index

In [ ]:
%%bash
PATH_SCRATCH='/central/scratch/btyeh'
PATH_INDEX="${PATH_SCRATCH}/index_hg38_mm10"

if [ ! -f "$PATH_INDEX"/hg38_mm10.fa.fai ]; then
    source ~/.bashrc
    conda activate chipdip
    
    cat "$PATH_SCRATCH"/hg38/*.fa "$PATH_SCRATCH"/mm10/*.fa > "$PATH_INDEX"/hg38_mm10.fa
    samtools faidx "$PATH_INDEX"/hg38_mm10.fa
fi

Download GENCODE annotations

In [ ]:
%%bash
set -e

PATH_SCRATCH='/central/scratch/btyeh'
PATH_ANNOT="$PATH_SCRATCH"/annot

if [ ! -f "$PATH_ANNOT"/hg38_mm10.gtf.gz.tbi ]; then
    source ~/.bashrc
    conda activate chipdip
    mkdir -p "$PATH_ANNOT"
    
    # download human GENCODE GTF and rename chromosomes
    wget -q -O - https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_46/gencode.v46.basic.annotation.gtf.gz |
        unpigz -p 4 -c |
        sort -k 1,1V -k 4,4V -k 5,5V |
        sed -E -e 's/^chr/h_chr/' > "$PATH_ANNOT"/hg38.gtf
    
    # download mouse GENCODE GTF and rename chromosomes
    wget -q -O - https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M25/gencode.vM25.basic.annotation.gtf.gz |
        unpigz -p 4 -c |
        sort -k 1,1V -k 4,4V -k 5,5V |
        sed -E -e 's/^chr/m_chr/' > "$PATH_ANNOT"/mm10.gtf
    
    # concatenate human and mouse genome GTFs
    cat "$PATH_ANNOT"/hg38.gtf "$PATH_ANNOT"/mm10.gtf |
        bgzip --threads 4 -c /dev/stdin > "$PATH_ANNOT"/hg38_mm10.gtf.gz
    
    rm "$PATH_ANNOT"/hg38.gtf "$PATH_ANNOT"/mm10.gtf
    
    # index the combined GTF
    tabix -f "$PATH_ANNOT"/hg38_mm10.gtf.gz
fi

Using paths to these annotation files, I created an IGV genome JSON file manually at `data_aux/20240625/hg38_mm10.json`

In [ ]:
with open(os.path.join(DIR_AUX, 'hg38_mm10.json'), 'rt') as f:
    print(json.dumps(json.load(f), indent=2))

# FastQC

In [ ]:
%%bash -s {DIR_DATA} {DIR_PROC} {reprocess}
DIR_DATA="$1"
DIR_PROC="$2"
reprocess="$3"
DIR_FASTQC_OUT="${DIR_PROC}/fastqc"
if [ ! -d "$DIR_FASTQC_OUT" ] || [ "$reprocess" = "True" ]; then
    mkdir -p "${DIR_PROC}/fastqc"
    cd "$DIR_DATA"
    source ~/.bashrc
    conda activate chipdip
    nohup fastqc *.fastq.gz -t 20 -q -o "$DIR_FASTQC_OUT" &> "${DIR_PROC}/fastqc.log" &
fi

# Data

In [ ]:
PATH_R1 = os.path.join(DIR_DATA, "HEK-pSM44_mixing_CTCF_H3K4me3_ChIP-seq_R1.fastq.gz")
PATH_R2 = os.path.join(DIR_DATA, "HEK-pSM44_mixing_CTCF_H3K4me3_ChIP-seq_R2.fastq.gz")

Final barcoded cell oligo structure: [Oligo (PC50_bc6 / PC50_bc7) + Odd + Even + Odd + Even + Odd + NYLigOdd](https://benchling.com/s/seq-JLjQnGxk3XhHueFGB5EQ?m=slm-x8jDl91Zo2EuS0W2OXJW)
- Top strand: 154-157 nt (variable due to NYLigOddStg)
- Bottom strand: 213-216 nt
- Library size: 297-300 bp
- Insert size for sequencing: 161-164 bp. A read 1 length of 121 bp would include the oligo barcode and all tags except the last Odd and NYLigOdd, while a read 2 length of 164 bp would include the entire oligo barcode.

Final barcoded DNA structure: [DPM + Odd + Even + Odd + Even + Odd + NYLigOdd](https://benchling.com/s/seq-MnXp2h0SAJ8dfB3N4lzc?m=slm-Copp8W7WpeZkiRtBICFG)
- Total added length (relative to genomic DNA): 308-311 bp (variable due to NYLigOddStg)
- Read 2 barcode length (including dA tail): 165 bp
- Read 1 DPM length: 10 bp

Sequencing
- Instrument and kit: AVITI Cloudbreak Freestyle
- Read 1 length: 120 bp
- Read 2 length: 180 bp

# Pipeline

1. Identify and trim bead barcode for all reads (add barcode to read name). Discard reads without a complete bead barcode.
2. Cell oligo processing
   - Identify species barcode
   - Deduplicate by species barcode and bead barcode
3. Chromatin processing
   - Trim DPM from chromatin reads, and check that DPM sequences match between Read 1 and Read 2
   - Align to human-mouse mixed genome: try aligning read 1 alone or paired-end alignment
   - Split by species to separate human and mouse BAM files
   - Remove reads (or read pairs) that overlap ENCODE blacklist regions
   - Deduplicate by alignment position and bead barcode
   - Generate bigWigs

Read counts

| Stage | Read count | Proportion of parent stage | Proportion of total |
| ----- | ---------: | -------------------------: | -------------------: |
| Total | 77,012,366 | | |
| `- Identifiable bead barcode` | 52,726,577 | 68.5% | 68.5% |
| `  - Oligo` | 40,021,181 | 75.9% | 52.0% |
| `    - Human (bc6)` | 19,581,303 |  |  |
| `    - Mouse (bc7)` | 20,439,878 |  |  |
| `  - Chromatin` |  |  |  |
| `    - CTCF` | 4,355,241 |  |  |
| `      - R1 aligned` |  |  |  |
| `        - Deduplicated` |  |  |  |
| `          - Human, not blacklisted` |  |  |  |
| `          - Mouse, not blacklisted` |  |  |  |
| `      - Paired-end aligned` |  |  |  |
| `        - Deduplicated` |  |  |  |
| `          - Human, not blacklisted` |  |  |  |
| `          - Mouse, not blacklisted` |  |  |  |
| `    - H3K4me3` | 4,001,469 |  |  |
| `      - R1 aligned` |  |  |  |
| `        - Deduplicated` |  |  |  |
| `          - Human, not blacklisted` |  |  |  |
| `          - Mouse, not blacklisted` |  |  |  |
| `      - Paired-end aligned` |  |  |  |
| `        - Deduplicated` |  |  |  |
| `          - Human, not blacklisted` |  |  |  |
| `          - Mouse, not blacklisted` |  |  |  |

In [ ]:
if os.path.exists(path_bead_counts) and not reprocess:
    df_beads = pd.DataFrame(
        data=np.load(path_bead_counts)['values'],
        index=pd.Index(np.load(path_bead_counts)['index'], name='bead'),
        columns=['human oligo', 'mouse oligo', 'human H3K4me3', 'mouse H3K4me3', 'human CTCF', 'mouse CTCF'],
    )

## Identify bead barcode from R2

`R[1|2]_bead-barcode.fastq.gz`
- Append `::bead=<bead number>` to read name
- Trim the bead barcode sequence from R2

In [ ]:
path_config_bead_barcode = os.path.join(DIR_AUX, 'splitcode_config-bead_barcode.tsv')

In [ ]:
%%bash -s {DIR_DATA} {DIR_PROC} {path_config_bead_barcode} {PATH_R1} {PATH_R2} {reprocess}
DIR_DATA="$1"
DIR_PROC="$2"
PATH_CONFIG="$3"
PATH_R1="$4"
PATH_R2="$5"
reprocess="$6"

PATH_MAPPING="$DIR_PROC/mapping_bead-barcode.tsv"
PATH_SUMMARY="$DIR_PROC/summary_bead-barcode.json"
PATH_OUT1="$DIR_PROC/R1_bead-barcode.fastq.gz"
PATH_OUT2="$DIR_PROC/R2_bead-barcode.fastq.gz"

if [ "$reprocess" = True ] || [ ! -f "${PATH_MAPPING}.gz" ]; then
    mkfifo pipe_R1 pipe_R2 pipe_mapping
    pids=()

    # modify read names so that there is no whitespace between the read name and bead barcode.
    sed -E -e 's/^(@\S+) BI:i:/\1::bead=/' pipe_R1 | pigz -p 4 > "$PATH_OUT1" &
    pids[0]="$!"
    sed -E -e 's/^(@\S+) BI:i:/\1::bead=/' pipe_R2 | pigz -p 4 > "$PATH_OUT2" &
    pids[1]="$!"
    cut -f 2,3 pipe_mapping | pigz -p 4 > "$PATH_MAPPING" &
    pids[2]="$!"
    
    splitcode -c "$PATH_CONFIG" \
        --nFastqs=2 --com-names --assign --no-outb -t 20 \
        --mapping=pipe_mapping --summary="$PATH_SUMMARY" --output=pipe_R1,pipe_R2 \
        "$PATH_R1" "$PATH_R2"

    for pid in "${pids[*]}"; do
        wait $pid
    done
    rm pipe_R1 pipe_R2
fi

Parse mapping into pandas DataFrame
- Columns: `Odd`, `Even`, `Odd2`, `Even2`, `Odd3`, `Y`, `Count`

In [ ]:
regex_bead_barcode = (
    'NYStgBot_(?P<Y>\d+),'
    'OddBot_(?P<Odd3>\d+),'
    'EvenBot_(?P<Even2>\d+),'
    'OddBot_(?P<Odd2>\d+),'
    'EvenBot_(?P<Even>\d+),'
    'OddBot_(?P<Odd>\d+)'
)
df_bead_barcodes = pd.read_csv(
    os.path.join(DIR_PROC, 'mapping_bead-barcode.tsv.gz'),
    sep='\t',
    names=['barcode', 'count'],
    dtype={'count': int}
).pipe(lambda df: pd.concat(
    (df, df['barcode'].str.extract(regex_bead_barcode).astype(np.uint8)),
    axis=1
))

In [ ]:
df_bead_barcodes.info()

In [ ]:
df_bead_barcodes['count'].sum()

### Verify indices of barcodes used each round

Actual indices used:
- Odd: rows A-D (indices 1-48)
- Even: rows A-D (indices 1-48)
- Odd2: rows E-H (indices 49-96)
- Even2: rows E-H (indices 49-96)
- Odd3: rows A-D (indices 1-48)
- NYLigOdd: rows E-H (indices 49-96)

In [ ]:
g = sns.catplot(
    kind='bar',
    data=(
        pd.concat(
            [
                df_bead_barcodes.groupby(r)['count'].sum().rename(r).reindex(pd.Index(list(range(1, 97))), fill_value=0)
                for r in ROUNDS
            ],
            axis=1
        )
        .reset_index(names='index')
        .melt(id_vars='index', var_name='round', value_name='count')
    ),
    x='index',
    y='count',
    row='round',
    row_order=ROUNDS,
    sharex=True,
    sharey=True,
    facet_kws=dict(gridspec_kws=dict(hspace=0.6))
)
g.axes.flatten()[-1].tick_params(axis='x', labelsize='xx-small')
g.figure.set_size_inches(10, 6)
g.savefig(os.path.join(DIR_RESULTS, 'tag indices used per round.png'), dpi=300, bbox_inches='tight')

Since `df_bead_barcodes` is a very large (>2 GB) variable, and it is not used later in this analysis, we delete it to free up memory.

In [ ]:
print('Size of df_bead_barcodes (in bytes):', sys.getsizeof(df_bead_barcodes))
del df_bead_barcodes
_ = gc.collect()

## Oligo read processing

1. Select only oligo read pairs
2. Validate the read pair
   - Since the oligo barcode (bc6 or bc7) is sequenced in both read 1 and read 2, make sure that it matches.
4. Generate table with following columns: bead #, human oligo count, mouse oligo count

In [ ]:
path_config_oligos = os.path.join(DIR_AUX, 'splitcode_config-oligos.tsv')

In [ ]:
%%bash -s {DIR_PROC} {path_config_oligos} {path_bead_counts} {reprocess}
DIR_PROC="$1"
PATH_CONFIG="$2"
PATH_BEAD_COUNTS="$3"
reprocess="$4"

PATH_R1="$DIR_PROC/R1_bead-barcode.fastq.gz"
PATH_R2="$DIR_PROC/R2_bead-barcode.fastq.gz"

if [ "$reprocess" = True ] || ([ ! -f "$PATH_BEAD_COUNTS" ] && [ ! -f "$DIR_PROC/oligos.csv.gz" ]); then
    # I would like to put the "keep" directive into the config file, but currently there is a bug in splitcode
    # parsing "remove" and "keep" directives in config files (https://github.com/pachterlab/splitcode/issues/33).
    # Consequently, I currently use the keep directive as a command line argument as a workaround
    splitcode -c "$PATH_CONFIG" \
        --nFastqs=2 --keep=<(echo -e "human,human_rc\nmouse,mouse_rc") --mod-names --select=0 --out-fasta -t 8 --pipe \
        "$PATH_R1" "$PATH_R2" |
        grep -E -e '^>' |
        sed -E \
            -e 's/^>.*::bead=([0-9]+)::\[(human|mouse)\]\[(human|mouse)_rc\]/\1,\2,\3/' \
            -e 's/human/h/g' -e 's/mouse/m/g' |
        sort |
        uniq -c |
        sed -E -e 's/^\s+([0-9]+)\s+/\1,/' |
        pigz -p 4 > "$DIR_PROC/oligos.csv.gz"
        # columns = count, bead, species, species2

    # in theory, could convert this table into binary format for greater speed and space efficiency
    # - for example, represent bead # and count values as unsigned 64-bit integers, and
    #   species and species2 as booleans (0 = human, 1 = mouse).
fi

In [ ]:
if 'df_beads' not in locals().keys() or reprocess:
    df_oligos = pd.read_csv(
        os.path.join(DIR_PROC, 'oligos.csv.gz'),
        sep=',',
        header=None,
        names=['count', 'bead', 'species', 'species2'],
        index_col='bead',
        dtype={
            'count': int,
            'bead': int,
            'species': DTYPE_SPECIES_ABBREV,
            'species2': DTYPE_SPECIES_ABBREV
        }
    )
    assert (df_oligos['species'] == df_oligos['species2']).all()
    df_oligos = (
        df_oligos
        .pivot(columns='species', values='count')
        .fillna(0)
        .astype(int)
        .rename(columns={'h': 'human', 'm': 'mouse'})
    )

In [ ]:
if 'df_beads' not in locals().keys() or reprocess:
    display(df_oligos.sum(axis=0))
else:
    display(df_beads[['human oligo', 'mouse oligo']].sum(axis=0))

## Chromatin read processing

In [ ]:
path_config_split_chromatin = os.path.join(DIR_AUX, 'splitcode_config-chromatin.tsv')
path_keep_chromatin = os.path.join(DIR_AUX, 'keep-chromatin.txt')

In [ ]:
%%bash -s {DIR_PROC} {path_config_split_chromatin} {path_keep_chromatin} {reprocess}
DIR_PROC="$1"
PATH_CONFIG="$2"
PATH_KEEP="$3"
reprocess="$4"

PATH_R1="$DIR_PROC/R1_bead-barcode.fastq.gz"
PATH_R2="$DIR_PROC/R2_bead-barcode.fastq.gz"

if [ "$reprocess" = True ] || [ ! -f "$DIR_PROC/CTCF_R1.fastq.gz" ]; then
    # change working directory to DIR_PROC so that the split files are generated in that directory
    cd "$DIR_PROC"

    splitcode -c "$PATH_CONFIG" \
        --nFastqs=2 -t 8 --keep="$PATH_KEEP" --keep-r1-r2 --gzip --no-output --no-outb \
        "$PATH_R1" "$PATH_R2"
fi

In [ ]:
%%bash -s {DIR_PROC} {DIR_AUX} {DIR_SCRIPTS}
DIR_PROC="$1"
DIR_AUX="$2"
DIR_SCRIPTS="$3"

source ~/.bashrc
conda activate snakemake

snakemake \
    --snakefile "${DIR_SCRIPTS}/Snakefile" \
    --directory "$DIR_PROC" \
    --configfile "${DIR_AUX}/config.yaml" \
    --dag |
dot -Tpdf > "${DIR_SCRIPTS}/dag.pdf"

snakemake \
    --snakefile "${DIR_SCRIPTS}/Snakefile" \
    --directory "$DIR_PROC" \
    --configfile "${DIR_AUX}/config.yaml" \
    --filegraph |
dot -Tpdf > "${DIR_SCRIPTS}/filegraph.pdf"

snakemake \
    --snakefile "${DIR_SCRIPTS}/Snakefile" \
    --directory "$DIR_PROC" \
    --configfile "${DIR_AUX}/config.yaml" \
    --rulegraph |
dot -Tpdf > "${DIR_SCRIPTS}/rulegraph.pdf"

In [ ]:
%%bash -s {DIR_PROC} {DIR_AUX} {DIR_SCRIPTS}
DIR_PROC="$1"
DIR_AUX="$2"
DIR_SCRIPTS="$3"

source ~/.bashrc
conda activate snakemake

snakemake \
    --snakefile "${DIR_SCRIPTS}/Snakefile" \
    --directory "$DIR_PROC" \
    --configfile "${DIR_AUX}/config.yaml" \
    --use-conda \
    --conda-frontend conda \
    --printshellcmds \
    --rerun-incomplete \
    -j 50 \
    --cluster-config "${DIR_AUX}/cluster.yaml" \
    --cluster "sbatch -c {cluster.cpus} \
    -t {cluster.time} -N {cluster.nodes} \
    --mem {cluster.mem} \
    --output {cluster.output} \
    --error {cluster.error}" \
    --cluster-cancel scancel \
    &> "${DIR_PROC}/snakemake.log"

# Analysis

Load filtered and deduplicated chromatin read alignment coordinates and counts.

In [ ]:
df_chromatin = []
for target in TARGETS:
    for species in SPECIES:
        for alignment_type in ALIGNMENT_TYPES:
            path_chromatin_counts = os.path.join(DIR_PROC, f'{target}-{alignment_type}_{species}_filtered_dedup_counts.bed.gz')
            df_chromatin.append(
                pd.read_csv(path_chromatin_counts, sep='\t', names=['chr', 'start', 'end', 'bead', 'count'])
                .assign(target=target, species=species, alignment_type=alignment_type)
                .astype({'target': DTYPE_TARGET, 'species': DTYPE_SPECIES, 'alignment_type': DTYPE_ALIGNMENT})
            )
df_chromatin = pd.concat(df_chromatin, axis=0, ignore_index=True).astype({'chr': 'category'})
df_chromatin['length'] = df_chromatin['end'] - df_chromatin['start']

Paired end vs. read 1-only alignment counts

In [ ]:
ax = sns.barplot(
    (
        df_chromatin
        .groupby(['target', 'species', 'alignment_type'], observed=True)
        .size()
        .rename('count')
        .reset_index()
        .pipe(lambda df: df.assign(**{'target-species': df['target'].str.cat(df['species'], sep='-')}))
    ),
    x='target-species',
    y='count',
    hue='alignment_type',
)
ax.set_title('Deduplicated reads per species, target, and alignment type')
ax.figure.savefig(
    os.path.join(DIR_RESULTS, 'paired-end vs. read 1-only alignment counts.png'),
    dpi=300,
    bbox_inches='tight'
)
ax.figure.show()

## Insert length distribution

Insert length distribution of aligned, properly paired reads after ENCODE blacklist filtering and deduplication.

In [ ]:
fig, axs = plt.subplots(2, 2, sharex=True, sharey='row', constrained_layout=True)

mask_PE = df_chromatin['alignment_type'] == 'PE'

# insert length by species
sns.ecdfplot(
    df_chromatin.loc[mask_PE],
    x='length',
    hue='species',
    ax=axs[0, 0]
)
sns.histplot(
    df_chromatin.loc[mask_PE],
    x='length',
    # auto binning can lead to artifactual periodic peaks in the histogram --> need to manually set binwidth
    # - can verify no actual periodic peaks by manually looking at values in the df_chromatin table
    binwidth=5,
    hue='species',
    ax=axs[1, 0]
)

# insert length by target
sns.ecdfplot(
    df_chromatin.loc[mask_PE],
    x='length',
    hue='target',
    ax=axs[0, 1]
)
sns.histplot(
    df_chromatin.loc[mask_PE],
    x='length',
    binwidth=5,
    hue='target',
    ax=axs[1, 1]
)

sns.move_legend(axs[0, 0], loc='lower right')
sns.move_legend(axs[0, 1], loc='lower right')
sns.move_legend(axs[1, 0], loc='upper right')
sns.move_legend(axs[1, 1], loc='upper right')

axs[1, 0].set_xlim(0, 1000)
axs[1, 0].set_xlabel('length (bp)')
axs[1, 0].set_ylabel('Count of deduplicated reads')
axs[0, 0].set_title('Insert length by species')
axs[0, 1].set_title('Insert length by target')

fig.suptitle('Chromatin fragmentation size')
fig.savefig(os.path.join(DIR_RESULTS, 'chromatin_fragmentation_size.png'), bbox_inches='tight', dpi=300)
fig.show()

## Visualize complexity estimates

Load complexity curves generated by preseq, and generate complexity curves using my own Poisson models

In [ ]:
df_chromatin_complexity_curves = []
df_chromatin_complexity_total = []
for target in TARGETS:
    for species in SPECIES:
        for alignment_type in ALIGNMENT_TYPES:

            # load complexity curves generated by preseq
            path_chromatin_complexity_curve = os.path.join(
                DIR_PROC,
                f'{target}-{alignment_type}_{species}_filtered_dedup_complexity-curve.txt'
            )
            path_chromatin_complexity_total = os.path.join(
                DIR_PROC,
                f'{target}-{alignment_type}_{species}_filtered_dedup_complexity-total.txt'
            )
            df_chromatin_complexity_curves.append(
                pd.read_csv(path_chromatin_complexity_curve, sep='\t', header=0)
                .rename(columns={
                    'TOTAL_READS': 'total_reads',
                    'EXPECTED_DISTINCT': 'expected_distinct',
                    'LOWER_0.95CI': 'lower_ci',
                    'UPPER_0.95CI': 'upper_ci'
                })
                .assign(target=target, species=species, alignment_type=alignment_type, model='preseq')
            )
            df_chromatin_complexity_total.append(
                pd.read_csv(path_chromatin_complexity_total, sep='\t', header=0)
                .assign(target=target, species=species, alignment_type=alignment_type, model='preseq')
                .squeeze()
            )

            # generate complexity curves using my own Poisson models
            mask = (df_chromatin['species'] == species) & \
                   (df_chromatin['target'] == target) & \
                   (df_chromatin['alignment_type'] == alignment_type)
            count_total = df_chromatin.loc[mask, 'count'].sum()
            count_mean = df_chromatin.loc[mask, 'count'].mean()
            M = int(estimate_library_complexity2(count_total, count_mean))
            df_chromatin_complexity_total.append(
                pd.Series(dict(
                    target=target,
                    species=species,
                    alignment_type=alignment_type,
                    model='zero-truncated Poisson',
                    pop_size_estimate=M
                ))
            )
            total_reads = np.linspace(1, int(3e7), 50, dtype=int)
            df_chromatin_complexity_curves.append(
                estimate_library_complexity_curve(total_reads, M)
                .assign(target=target, species=species, alignment_type=alignment_type, model='Poisson')
            )

df_chromatin_complexity_curves = (
    pd.concat(df_chromatin_complexity_curves, axis=0, ignore_index=True)
    .astype({'target': DTYPE_TARGET, 'species': DTYPE_SPECIES, 'alignment_type': DTYPE_ALIGNMENT, 'model': 'category'})
)
df_chromatin_complexity_total = (
    pd.concat(df_chromatin_complexity_total, axis=1, ignore_index=True).T
    .astype({'target': DTYPE_TARGET, 'species': DTYPE_SPECIES, 'alignment_type': DTYPE_ALIGNMENT, 'model': 'category'})
)

Plot complexity curves, split by alignment type.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 4), sharex=True, sharey=True, constrained_layout=True)
axs[0].set_title('Alignment: paired end')
axs[0].set_xlabel('Total reads')
axs[0].set_ylabel('Expected unique reads')
axs[1].set_title('Alignment: read 1 only')   

def filter_df(df, filter_dict):
    mask = np.ones(len(df), dtype=bool)
    for col, value in filter_dict.items():
        if col in df.columns:
            mask &= (df[col] == value)
    return df.loc[mask]

i = 0
for target in TARGETS:
    for species in SPECIES:
        color = f'C{i}'
        i += 1
        for model, ls in zip(('preseq', 'Poisson'), ('solid', 'dotted')):
            mask_dict = dict(target=target, species=species, model=model, alignment_type='PE')
            kwargs = dict(color=color, ls=ls, label=f'{target}, {species} ({model})')
            axs[0].plot(
                filter_df(df_chromatin_complexity_curves, mask_dict)['total_reads'],
                filter_df(df_chromatin_complexity_curves, mask_dict)['expected_distinct'],
                **kwargs
            )
            mask_dict = dict(target=target, species=species, model=model, alignment_type='R1')
            axs[1].plot(
                filter_df(df_chromatin_complexity_curves, mask_dict)['total_reads'],
                filter_df(df_chromatin_complexity_curves, mask_dict)['expected_distinct'],
                **kwargs
            )
            if model == 'Poisson':
                kwargs = dict(color=color, ls='dashed', label=f'{target}, {species} (current reads)')
                axs[0].axvline(
                    filter_df(df_chromatin, mask_dict)['count'].sum(),
                    **kwargs
                )
                axs[1].axvline(
                    filter_df(df_chromatin, mask_dict)['count'].sum(),
                    **kwargs
                )
axs[1].legend(loc='upper left', bbox_to_anchor=(1, 1))
fig.suptitle('Complexity curves')
fig.savefig(
    os.path.join(DIR_RESULTS, 'chromatin complexity curves.png'),
    dpi=300,
    bbox_inches='tight'
)
fig.show()

Calculate estimated proportion of library complexity sequenced, based on paired-end alignments.

In [ ]:
fig, ax = plt.subplots()

x = 1
xlabels = []
for target in TARGETS:
    for species in SPECIES:
        xlabels.append(f'{target}-{species}')
        mask_dict = dict(target=target, species=species, alignment_type='PE', model='preseq')
        complexity_estimate = filter_df(df_chromatin_complexity_total, mask_dict).squeeze()['pop_size_estimate']
        complexity_observed = len(filter_df(df_chromatin, mask_dict))
        ax.bar(x, complexity_observed, facecolor=f'C{x-1}')
        ax.bar(x, complexity_estimate, facecolor=(0, 0, 0, 0), edgecolor=f'C{x-1}', linewidth=2, ls='dotted')
        ax.text(
            x,
            complexity_observed + 0.2e6,
            f'{complexity_observed / complexity_estimate:.1%}',
            ha='center'
        )
        x += 1
ax.set_xticks([1, 2, 3, 4], xlabels)
ax.set_title('Proportion of estimated total chromatin molecules sequenced')

legend_elements = [
    matplotlib.patches.Patch(facecolor='black', label='sequenced'),
    matplotlib.lines.Line2D([0], [0], color='black', lw=2, ls='dotted', label='estimated complexity')
]
ax.legend(handles=legend_elements, loc='best')

fig.savefig(
    os.path.join(DIR_RESULTS, 'proportion chromatin sequenced.png'),
    dpi=300,
    bbox_inches='tight'
)
fig.show()

## Verify that alignment to combined human-mouse genome correctly distinguished chromatin species

Note: I only perform this analysis on paired-end alignments, not on read 1-only alignments.

Conclusion: alignment to the combined genome largely identified the correct species.
- For >99.9% of read pairs, the alignment to the identified species genome gave an alignment score >= that of aligning to the combined genome, whereas alignment to the other species genome gave worse alignments.

Extract MAPQ and alignment scores (`AS:i:[#]` SAM tag) into table with columns:
- aligned species using combined genome
- protein target (CTCF or H3K4me3)
- R1 MAPQ, combined genome
- R2 MAPQ, combined genome
- R1 MAPQ, human genome
- R2 MAPQ, human genome
- R1 MAPQ, mouse genome
- R2 MAPQ, mouse genome
- R1 AS, combined genome
- R2 AS, combined genome
- R1 AS, combined genome
- R2 AS, combined genome
- R1 AS, combined genome
- R2 AS, combined genome

In [ ]:
def match_r1_r2(
    ref_r1: pysam.AlignedSegment,
    ref_r2: pysam.AlignedSegment,
    new_r1: pysam.AlignedSegment,
    new_r2: pysam.AlignedSegment
):
    '''
    Given the alignment of two read pairs of a query template (one pair as a reference alignment,
    one pair as a new alignment), return reorder the reads from the new read pair (if necessary)
    to match the order of reads from the reference alignment pair.

    Approach
    - Compare read sequence
    - Compare read alignment orientations
    '''
    assert all(x.query_name == ref_r1.query_name for x in (ref_r2, new_r1, new_r2))
    ref_seqs = (ref_r1.get_forward_sequence(), ref_r2.get_forward_sequence())
    new_seqs = (new_r1.get_forward_sequence(), new_r2.get_forward_sequence())
    if new_seqs[0] == new_seqs[1]:
        # if read 1 and read 2 sequences are the same:
        # - if both reads are unmapped, then the order is irrelevant
        # - if only 1 read is mapped, then match it to the reference read 1 or read 2 by orientation
        # - if both reads are mapped, then directly compare the alignments to the reference reads
        if new_r1.is_unmapped and new_r2.is_unmapped:
            return (new_r1, new_r2)
        elif new_r1.is_unmapped:
            if new_r2.is_reverse == ref_r1.is_reverse:
                return (new_r2, new_r1)
            else:
                return (new_r1, new_r2)
        elif new_r2.is_unmapped:
            if new_r1.is_reverse == ref_r1.is_reverse:
                return (new_r1, new_r2)
            else:
                return (new_r2, new_r1)
        else:
            if ref_r1.compare(new_r1) == 0:
                return (new_r1, new_r2)
            elif ref_r1.compare(new_r2) == 0:
                return (new_r2, new_r1)
            if new_r1.is_reverse and new_r2.is_reverse:
                # no implementation yet for this edge case where read 1 and read 2 sequences are identical,
                # they are both mapped using the same orientation, and their alignments are different from
                # the reference alignments
                raise ValueError
    if new_seqs == ref_seqs:
        # based on matching the read sequences, the ordering of the new read pair was already correct
        return new_r1, new_r2
    elif (new_seqs[1], new_seqs[0]) == ref_seqs:
        # based on matching the read sequences, the ordering of the new read pair was flipped
        return new_r2, new_r1
    else:
        # the code should never get here, which would mean that the new read sequences
        # do not match the reference read sequences
        raise ValueError

In [ ]:
df_scores = []
for target in TARGETS:
    for species in SPECIES:
        scores = []
        # these are all name-sorted BAM files
        path_bam_combined = os.path.join(DIR_PROC, f'{target}-PE_{species}_filtered_dedup_sort-name.bam')
        path_bam_human = os.path.join(DIR_PROC, f'{target}-PE_{species}_realign-human.bam')
        path_bam_mouse = os.path.join(DIR_PROC, f'{target}-PE_{species}_realign-mouse.bam')
        with pysam.AlignmentFile(path_bam_combined, 'rb', threads=n_proc) as fc, \
             pysam.AlignmentFile(path_bam_human, 'rb', threads=n_proc) as fh, \
             pysam.AlignmentFile(path_bam_mouse, 'rb', threads=n_proc) as fm:
            for (r1c, r2c), (r1h, r2h), (r1m, r2m) in tqdm(zip(
                grouper(fc.fetch(until_eof=True), 2, incomplete='strict'),
                grouper(fh.fetch(until_eof=True), 2, incomplete='strict'),
                grouper(fm.fetch(until_eof=True), 2, incomplete='strict'))):
                r1h, r2h = match_r1_r2(r1c, r2c, r1h, r2h)
                r1m, r2m = match_r1_r2(r1c, r2c, r1m, r2m)
                scores.append(dict(
                    MAPQ_combined_R1=r1c.mapping_quality,
                    MAPQ_combined_R2=r2c.mapping_quality,
                    MAPQ_human_R1=r1h.mapping_quality,
                    MAPQ_human_R2=r2h.mapping_quality,
                    MAPQ_mouse_R1=r1m.mapping_quality,
                    MAPQ_mouse_R2=r2m.mapping_quality,
                    AS_combined_R1=r1c.get_tag('AS') if r1c.has_tag('AS') else pd.NA,
                    AS_combined_R2=r2c.get_tag('AS') if r2c.has_tag('AS') else pd.NA,
                    AS_human_R1=r1h.get_tag('AS') if r1h.has_tag('AS') else pd.NA,
                    AS_human_R2=r2h.get_tag('AS') if r2h.has_tag('AS') else pd.NA,
                    AS_mouse_R1=r1m.get_tag('AS') if r1m.has_tag('AS') else pd.NA,
                    AS_mouse_R2=r2m.get_tag('AS') if r2m.has_tag('AS') else pd.NA,
                ))
        scores = (
            pd.DataFrame(scores)
            .astype(pd.Int64Dtype())
            .assign(target=target, species=species)
            .astype(dict(target=DTYPE_TARGET, species=DTYPE_SPECIES))
        )
        df_scores.append(scores)
df_scores = pd.concat(df_scores, axis=0, ignore_index=True)
del scores
gc.collect()
df_scores.info()

For read pairs aligned (via the combined genome) to **human chromosomes**, visualize relationship between **alignment scores** on the combined genome versus individual species genomes. Here, I calculate the alignment score for the read pair as the sum of alignment scores of read 1 and read 2.

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(8, 5), sharex=True, sharey=True, constrained_layout=True)
mask_human_CTCF = (df_scores['species'] == 'human') & (df_scores['target'] == 'CTCF')
mask_human_H3K4me3 = (df_scores['species'] == 'human') & (df_scores['target'] == 'H3K4me3')

axs[0, 0].set_title('CTCF ChIP')
axs[0, 0] = sns.histplot(
    (
        df_scores.loc[mask_human_CTCF]
        .pipe(lambda df: df.assign(**{
            'combined genomes': df['AS_combined_R1'] + df['AS_combined_R2'],
            'human genome': df['AS_human_R1'] + df['AS_human_R2']
        }))
    ),
    x='combined genomes',
    y='human genome',
    bins=30,
    discrete=(True, True),
    cbar=True, cbar_kws=dict(shrink=1, format="%.e"),
    ax=axs[0, 0]
)
axs[1, 0] = sns.histplot(
    (
        df_scores.loc[mask_human_CTCF]
        .pipe(lambda df: df.assign(**{
            'combined genomes': df['AS_combined_R1'] + df['AS_combined_R2'],
            'mouse genome': df['AS_mouse_R1'] + df['AS_mouse_R2']
        }))
    ),
    x='combined genomes',
    y='mouse genome',
    bins=30,
    discrete=(True, True),
    cbar=True, cbar_kws=dict(shrink=1, format="%.e"),
    ax=axs[1, 0]
)

axs[0, 1].set_title('H3K4me3 ChIP')
axs[0, 1] = sns.histplot(
    (
        df_scores.loc[mask_human_H3K4me3]
        .pipe(lambda df: df.assign(**{
            'combined genomes': df['AS_combined_R1'] + df['AS_combined_R2'],
            'human genome': df['AS_human_R1'] + df['AS_human_R2']
        }))
    ),
    x='combined genomes',
    y='human genome',
    bins=30,
    discrete=(True, True),
    cbar=True, cbar_kws=dict(shrink=1, format="%.e"),
    ax=axs[0, 1]
)
axs[1, 1] = sns.histplot(
    (
        df_scores.loc[mask_human_H3K4me3]
        .pipe(lambda df: df.assign(**{
            'combined genomes': df['AS_combined_R1'] + df['AS_combined_R2'],
            'mouse genome': df['AS_mouse_R1'] + df['AS_mouse_R2']
        }))
    ),
    x='combined genomes',
    y='mouse genome',
    bins=30,
    discrete=(True, True),
    cbar=True, cbar_kws=dict(shrink=1, format="%.e"),
    ax=axs[1, 1]
)

fig.suptitle('Distribution of paired-end alignment scores of human-assigned reads')
fig.savefig(
    os.path.join(DIR_RESULTS, 'paired-end alignment scores, human.png'),
    dpi=300,
    bbox_inches='tight'
)
fig.show()

For read pairs aligned (via the combined genome) to **mouse chromosomes**, visualize relationship between **alignment scores** on the combined genome versus individual species genomes. Here, I calculate the alignment score for the read pair as the sum of alignment scores of read 1 and read 2.

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(8, 5), sharex=True, sharey=True, constrained_layout=True)
mask_mouse_CTCF = (df_scores['species'] == 'mouse') & (df_scores['target'] == 'CTCF')
mask_mouse_H3K4me3 = (df_scores['species'] == 'mouse') & (df_scores['target'] == 'H3K4me3')

axs[0, 0].set_title('CTCF ChIP')
axs[0, 0] = sns.histplot(
    (
        df_scores.loc[mask_mouse_CTCF]
        .pipe(lambda df: df.assign(**{
            'combined genomes': df['AS_combined_R1'] + df['AS_combined_R2'],
            'human genome': df['AS_human_R1'] + df['AS_human_R2']
        }))
    ),
    x='combined genomes',
    y='human genome',
    bins=30,
    discrete=(True, True),
    cbar=True, cbar_kws=dict(shrink=1, format="%.e"),
    ax=axs[0, 0]
)
axs[1, 0] = sns.histplot(
    (
        df_scores.loc[mask_mouse_CTCF]
        .pipe(lambda df: df.assign(**{
            'combined genomes': df['AS_combined_R1'] + df['AS_combined_R2'],
            'mouse genome': df['AS_mouse_R1'] + df['AS_mouse_R2']
        }))
    ),
    x='combined genomes',
    y='mouse genome',
    bins=30,
    discrete=(True, True),
    cbar=True, cbar_kws=dict(shrink=1, format="%.e"),
    ax=axs[1, 0]
)

axs[0, 1].set_title('H3K4me3 ChIP')
axs[0, 1] = sns.histplot(
    (
        df_scores.loc[mask_mouse_H3K4me3]
        .pipe(lambda df: df.assign(**{
            'combined genomes': df['AS_combined_R1'] + df['AS_combined_R2'],
            'human genome': df['AS_human_R1'] + df['AS_human_R2']
        }))
    ),
    x='combined genomes',
    y='human genome',
    bins=30,
    discrete=(True, True),
    cbar=True, cbar_kws=dict(shrink=1, format="%.e"),
    ax=axs[0, 1]
)
axs[1, 1] = sns.histplot(
    (
        df_scores.loc[mask_mouse_H3K4me3]
        .pipe(lambda df: df.assign(**{
            'combined genomes': df['AS_combined_R1'] + df['AS_combined_R2'],
            'mouse genome': df['AS_mouse_R1'] + df['AS_mouse_R2']
        }))
    ),
    x='combined genomes',
    y='mouse genome',
    bins=30,
    discrete=(True, True),
    cbar=True, cbar_kws=dict(shrink=1, format="%.e"),
    ax=axs[1, 1]
)

fig.suptitle('Distribution of paired-end alignment scores of mouse-assigned reads')
fig.savefig(
    os.path.join(DIR_RESULTS, 'paired-end alignment scores, mouse.png'),
    dpi=300,
    bbox_inches='tight'
)
fig.show()

For read pairs aligned (via the combined genome) to **human chromosomes**, visualize relationship between **mapping quality (MAPQ) scores** on the combined genome versus individual species genomes. Here, I calculate the mapping quality score for the read pair as the sum of mapping quality scores of read 1 and read 2.

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(8, 5), sharex=True, sharey=True, constrained_layout=True)
mask_human_CTCF = (df_scores['species'] == 'human') & (df_scores['target'] == 'CTCF')
mask_human_H3K4me3 = (df_scores['species'] == 'human') & (df_scores['target'] == 'H3K4me3')

axs[0, 0].set_title('CTCF ChIP')
axs[0, 0] = sns.histplot(
    (
        df_scores.loc[mask_human_CTCF]
        .pipe(lambda df: df.assign(**{
            'combined genomes': df['MAPQ_combined_R1'] + df['MAPQ_combined_R2'],
            'human genome': df['MAPQ_human_R1'] + df['MAPQ_human_R2']
        }))
    ),
    x='combined genomes',
    y='human genome',
    bins=30,
    discrete=(True, True),
    cbar=True, cbar_kws=dict(shrink=1, format="%.e"),
    ax=axs[0, 0]
)
axs[1, 0] = sns.histplot(
    (
        df_scores.loc[mask_human_CTCF]
        .pipe(lambda df: df.assign(**{
            'combined genomes': df['MAPQ_combined_R1'] + df['MAPQ_combined_R2'],
            'mouse genome': df['MAPQ_mouse_R1'] + df['MAPQ_mouse_R2']
        }))
    ),
    x='combined genomes',
    y='mouse genome',
    bins=30,
    discrete=(True, True),
    cbar=True, cbar_kws=dict(shrink=1, format="%.e"),
    ax=axs[1, 0]
)

axs[0, 1].set_title('H3K4me3 ChIP')
axs[0, 1] = sns.histplot(
    (
        df_scores.loc[mask_human_H3K4me3]
        .pipe(lambda df: df.assign(**{
            'combined genomes': df['MAPQ_combined_R1'] + df['MAPQ_combined_R2'],
            'human genome': df['MAPQ_human_R1'] + df['MAPQ_human_R2']
        }))
    ),
    x='combined genomes',
    y='human genome',
    bins=30,
    discrete=(True, True),
    cbar=True, cbar_kws=dict(shrink=1, format="%.e"),
    ax=axs[0, 1]
)
axs[1, 1] = sns.histplot(
    (
        df_scores.loc[mask_human_H3K4me3]
        .pipe(lambda df: df.assign(**{
            'combined genomes': df['MAPQ_combined_R1'] + df['MAPQ_combined_R2'],
            'mouse genome': df['MAPQ_mouse_R1'] + df['MAPQ_mouse_R2']
        }))
    ),
    x='combined genomes',
    y='mouse genome',
    bins=30,
    discrete=(True, True),
    cbar=True, cbar_kws=dict(shrink=1, format="%.e"),
    ax=axs[1, 1]
)

fig.suptitle('Distribution of paired-end MAPQ scores of human-assigned reads')
fig.savefig(
    os.path.join(DIR_RESULTS, 'paired-end MAPQ scores, human.png'),
    dpi=300,
    bbox_inches='tight'
)
fig.show()

For read pairs aligned (via the combined genome) to **mouse chromosomes**, visualize relationship between **mapping quality (MAPQ) scores** on the combined genome versus individual species genomes. Here, I calculate the mapping quality score for the read pair as the sum of mapping quality scores of read 1 and read 2.

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(8, 5), sharex=True, sharey=True, constrained_layout=True)
mask_mouse_CTCF = (df_scores['species'] == 'mouse') & (df_scores['target'] == 'CTCF')
mask_mouse_H3K4me3 = (df_scores['species'] == 'mouse') & (df_scores['target'] == 'H3K4me3')

axs[0, 0].set_title('CTCF ChIP')
axs[0, 0] = sns.histplot(
    (
        df_scores.loc[mask_mouse_CTCF]
        .pipe(lambda df: df.assign(**{
            'combined genomes': df['MAPQ_combined_R1'] + df['MAPQ_combined_R2'],
            'human genome': df['MAPQ_human_R1'] + df['MAPQ_human_R2']
        }))
    ),
    x='combined genomes',
    y='human genome',
    bins=30,
    discrete=(True, True),
    cbar=True, cbar_kws=dict(shrink=1, format="%.e"),
    ax=axs[0, 0]
)
axs[1, 0] = sns.histplot(
    (
        df_scores.loc[mask_mouse_CTCF]
        .pipe(lambda df: df.assign(**{
            'combined genomes': df['MAPQ_combined_R1'] + df['MAPQ_combined_R2'],
            'mouse genome': df['MAPQ_mouse_R1'] + df['MAPQ_mouse_R2']
        }))
    ),
    x='combined genomes',
    y='mouse genome',
    bins=30,
    discrete=(True, True),
    cbar=True, cbar_kws=dict(shrink=1, format="%.e"),
    ax=axs[1, 0]
)

axs[0, 1].set_title('H3K4me3 ChIP')
axs[0, 1] = sns.histplot(
    (
        df_scores.loc[mask_mouse_H3K4me3]
        .pipe(lambda df: df.assign(**{
            'combined genomes': df['MAPQ_combined_R1'] + df['MAPQ_combined_R2'],
            'human genome': df['MAPQ_human_R1'] + df['MAPQ_human_R2']
        }))
    ),
    x='combined genomes',
    y='human genome',
    bins=30,
    discrete=(True, True),
    cbar=True, cbar_kws=dict(shrink=1, format="%.e"),
    ax=axs[0, 1]
)
axs[1, 1] = sns.histplot(
    (
        df_scores.loc[mask_mouse_H3K4me3]
        .pipe(lambda df: df.assign(**{
            'combined genomes': df['MAPQ_combined_R1'] + df['MAPQ_combined_R2'],
            'mouse genome': df['MAPQ_mouse_R1'] + df['MAPQ_mouse_R2']
        }))
    ),
    x='combined genomes',
    y='mouse genome',
    bins=30,
    discrete=(True, True),
    cbar=True, cbar_kws=dict(shrink=1, format="%.e"),
    ax=axs[1, 1]
)

fig.suptitle('Distribution of paired-end MAPQ scores of mouse-assigned reads')
fig.savefig(
    os.path.join(DIR_RESULTS, 'paired-end MAPQ scores, mouse.png'),
    dpi=300,
    bbox_inches='tight'
)
fig.show()

Proportion of read pairs with lower alignment scores (summed across read 1 and read 2) when aligned to their assigned genome than aligned to the combined genome.

In [ ]:
prop_lower_score = []
for species in SPECIES:
    mask_denom = (df_scores['species'] == species)
    mask_num = mask_denom & \
        ((df_scores[f'AS_{species}_R1'] + df_scores[f'AS_{species}_R2'] < df_scores['AS_combined_R1'] + df_scores['AS_combined_R2']) |
         (df_scores[f'AS_{species}_R1'].isna() | df_scores[f'AS_{species}_R2'].isna()))
    prop_lower_score.append({'species': species, 'lower score count': mask_num.sum(), 'total': mask_denom.sum(), 'new genome': species})
    species2 = 'mouse' if species == 'human' else 'human'
    mask_num2 = mask_denom & \
        ((df_scores[f'AS_{species2}_R1'] + df_scores[f'AS_{species2}_R2'] < df_scores['AS_combined_R1'] + df_scores['AS_combined_R2']) |
         (df_scores[f'AS_{species2}_R1'].isna() | df_scores[f'AS_{species2}_R2'].isna()))
    prop_lower_score.append({'species': species, 'lower score count': mask_num2.sum(), 'total': mask_denom.sum(), 'new genome': species2})
prop_lower_score = pd.DataFrame(prop_lower_score)
prop_lower_score['proportion'] = prop_lower_score['lower score count'] / prop_lower_score['total']

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(6, 5), sharey=True, constrained_layout=True)
sns.barplot(
    prop_lower_score.loc[prop_lower_score['species'] == prop_lower_score['new genome']],
    x='species',
    y='proportion',
    ax=axs[0]
)
axs[0].set_ylabel('\n'.join((
    'proportion of read pairs with lower alignment scores',
    'to species-specific genome than to combined genome'
)))
axs[0].set_xlabel('species assigned\nand used for realignment')

sns.barplot(
    prop_lower_score.loc[prop_lower_score['species'] != prop_lower_score['new genome']],
    x='species',
    y='proportion',
    ax=axs[1]
)
axs[1].set_xlabel('species used for realignment\n(opposite assigned)')

for ax in axs:
    for container in ax.containers:
        ax.bar_label(container, fmt='{:.3%}')

fig.savefig(
    os.path.join(DIR_RESULTS, 'proportion of read pairs with lower alignment scores.png'),
    dpi=300,
    bbox_inches='tight'
)
fig.show()

**For a read pair that Bowtie 2 aligned to a chromosome of species X in the combined genome, why would the read pair yield a lower alignment score when aligned directly to the genome of species X?**

If the read pair truly came from species X, one possibility is that when read seeds were aligned to the combined genome, the number of seed hits exceeded the "repetitive seeds" threshold, prompting Bowtie 2 to use different (perhaps more optimal) seeds that eventually led to a better alignment to the species X genome. The initial seeds never exceeded the "repetitive seeds" threshold when aligned only against the species X genome, such that the more optimal seeds were never used.

## Molecules per bead

### Beads

In [ ]:
path_bead_counts = os.path.join(DIR_PROC, 'bead_counts.npz')

Total number of bead barcodes identified.

In [ ]:
!unpigz -c {os.path.join(DIR_PROC, 'mapping_bead-barcode.tsv.gz')} | wc -l

Number of beads with oligos

In [ ]:
if 'df_beads' not in locals().keys() or reprocess:
    display(len(df_oligos))
else:
    display(((df_beads['human oligo'] > 0) | (df_beads['mouse oligo'] > 0)).sum())

Number of beads with aligned, species-specific, non-blacklisted chromatin

In [ ]:
len(df_chromatin['bead'].unique())

Create table of oligo and chromatin counts for each bead.
- The chromatin counts are deduplicated.
- The oligo counts are not, since they do not have UMIs.

In [ ]:
if not os.path.exists(path_bead_counts) or reprocess:
    df_chromatin_counts = (
        df_chromatin
        .loc[df_chromatin['alignment_type'] == 'PE']
        .groupby(['bead', 'target', 'species'], observed=True)
        .size()
        .rename('count')
        .reset_index()
        .pivot(index='bead', columns=['species', 'target'], values='count')
        .fillna(0)
        .astype(int)
    )
    df_chromatin_counts.columns = [' '.join(col).strip() for col in df_chromatin_counts.columns.values]

    df_beads = (
        df_oligos
        .rename(columns={'human': 'human oligo', 'mouse': 'mouse oligo'})
        .join(df_chromatin_counts, how='outer')
        .fillna(0)
        .astype(np.int32)
    )

    # because of its sparsity, saving this DataFrame in a compressed .npz format saves significant space
    np.savez_compressed(
        path_bead_counts,
        index=df_beads.index.values,
        values=df_beads.values
    )
else:
    df_beads = pd.DataFrame(
        data=np.load(path_bead_counts)['values'],
        index=pd.Index(np.load(path_bead_counts)['index'], name='bead'),
        columns=['human oligo', 'mouse oligo', 'human H3K4me3', 'mouse H3K4me3', 'human CTCF', 'mouse CTCF'],
    )

In [ ]:
df_beads.info()

Sparsity (proportion of values that are zeros)

In [ ]:
(df_beads.values == 0).sum().sum() / (15519769 * 6)

### Bivariate distribution of oligos and chromatin per bead

### Chromatin per bead

In [ ]:
# index = bead
# columns = human (proportion), mouse (proportion), total (count), chromatin molecules per bead (binned count)
df_chromatin_species_per_bead = (
    df_chromatin
    .loc[df_chromatin['alignment_type'] == 'PE']
    .groupby(['bead', 'species'], observed=True)
    .size()
    .rename('count')
    .reset_index()
    .pivot(index='bead', values='count', columns='species')
    .fillna(0)
    .astype(int)
    .pipe(lambda df: df.assign(total=df['human'] + df['mouse']))
    .pipe(lambda df: df.assign(human=df['human']/df['total'], mouse=df['mouse']/df['total']))
)

df_chromatin_species_per_bead = df_chromatin_species_per_bead.assign(**{
    'chromatin molecules per bead': pd.cut(
        df_chromatin_species_per_bead['total'],
        bins=[0, 1, 4, 16, 64, 256, 1024, np.inf],
        right=True,
        labels=['1', '2-4', '5-16', '17-64', '65-256', '257-1024', '1025+']
    )})

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 6), sharex=True, constrained_layout=True)
sns.ecdfplot(df_chromatin_species_per_bead, x='total', log_scale=True, ax=axs[0, 0])
axs[0, 0].set_ylabel('proportion of beads')
sns.ecdfplot(df_chromatin_species_per_bead, x='total', weights='total', log_scale=True, ax=axs[0, 1])
axs[0, 1].set_ylabel('proportion of chromatin reads')
sns.histplot(df_chromatin_species_per_bead, x='total', bins=40, log_scale=(True, True), ax=axs[1, 0])
axs[1, 0].set_ylabel('number of beads')
sns.histplot(df_chromatin_species_per_bead, x='total', weights='total', bins=40, log_scale=(True, True), ax=axs[1, 1])
axs[1, 1].set_ylabel('number of chromatin reads')
axs[1, 0].set_xlabel(None)
fig.supxlabel('number of chromatin molecules per bead')
fig.suptitle('Distribution of chromatin molecules per bead\n(on chromatin-containing beads)')
fig.savefig(
    os.path.join(DIR_RESULTS, 'chromatin molecules per bead.png'),
    dpi=300,
    bbox_inches='tight'
)
fig.show()

In [ ]:
fig, axs = plt.subplots(2, 1, sharex=True, constrained_layout=True)
sns.histplot(
    df_chromatin_species_per_bead.loc[df_chromatin_species_per_bead['total'] > 1]
    .pipe(lambda df: df.assign(**{
        'chromatin molecules per bead': df['chromatin molecules per bead'].cat.remove_categories('1')
    })),
    x='human',
    stat='proportion',
    multiple='stack',
    common_norm=False,
    hue='chromatin molecules per bead',
    palette='viridis',
    ax=axs[0]
)
sns.ecdfplot(
    df_chromatin_species_per_bead.loc[df_chromatin_species_per_bead['total'] > 1]
    .pipe(lambda df: df.assign(**{
        'chromatin molecules per bead': df['chromatin molecules per bead'].cat.remove_categories('1')
    })),
    x='human',
    hue='chromatin molecules per bead',
    palette='viridis',
    legend=False,
    ax=axs[1]
)
axs[0].set_ylabel('proportion of beads')
axs[1].set_ylabel('proportion of beads')
axs[1].set_xlabel('proportion of chromatin molecules on a bead\nfrom human genome (vs. mouse genome)')
axs[0].set_title('Distribution of chromatin species on beads')
fig.savefig(
    os.path.join(DIR_RESULTS, 'chromatin species on beads.png'),
    dpi=300,
    bbox_inches='tight'
)
fig.show()

#### CTCF vs. H3K4me3

#### Human vs. mouse

### Oligos per bead

In [ ]:
df_oligo_species_per_bead = (
    df_oligos
    .assign(total=df_oligos.sum(axis=1))
    .pipe(lambda df: df.assign(**{
        'human': df['human'] / df['total'],
        'mouse': df['mouse'] / df['total']
    }))
)

df_oligo_species_per_bead = df_oligo_species_per_bead.assign(**{
    'oligo molecules per bead': pd.cut(
        df_oligo_species_per_bead['total'],
        bins=[0, 1, 4, 16, 64, 256, 1024, np.inf],
        right=True,
        labels=['1', '2-4', '5-16', '17-64', '65-256', '257-1024', '1025+']
    )})

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 6), sharex=True, constrained_layout=True)

df_oligos_long = (
    df_oligos
    .assign(total=df_oligos.sum(axis=1))
    .melt(ignore_index=False, var_name='oligo species', value_name='count')
)

xscale = matplotlib.scale.SymmetricalLogScale(None, base=10, linthresh=1, linscale=0.1)
symlog_transform = xscale.get_transform()
symlog_transform_inverse = symlog_transform.inverted()
bins = symlog_transform_inverse.transform(np.linspace(
    symlog_transform.transform(-0.6),
    symlog_transform.transform(df_oligos_long['count'].max()),
    30
))

# re-implementing the == operator of Numpy array to bypass a seaborn bug in sns.histplot
# when using weights and user-specific bins
# https://github.com/mwaskom/seaborn/issues/3801
class myclass(np.ndarray):
    def __eq__(self, other):
        if type(other) is str and other == 'auto':
            return False
        else:
            return super().__eq__(other)
bins = myclass(bins.shape, buffer=bins, dtype=float)

sns.ecdfplot(
    df_oligos_long,
    x='count',
    hue='oligo species',
    ax=axs[0, 0]
)
axs[0, 0].set_ylabel('proportion of beads')
sns.move_legend(axs[0, 0], loc='lower right')

sns.ecdfplot(
    df_oligos_long,
    x='count',
    weights='count',
    hue='oligo species',
    ax=axs[0, 1]
)
axs[0, 1].set_ylabel('proportion of oligo reads')
sns.move_legend(axs[0, 1], loc='lower right')

sns.histplot(
    df_oligos_long,
    x='count',
    hue='oligo species',
    bins=bins,
    alpha=0.3,
    ax=axs[1, 0]
)
axs[1, 0].set_yscale('symlog', linthresh=10)
axs[1, 0].set_ylabel('number of beads')
axs[1, 0].set_xlabel(None)
sns.move_legend(axs[1, 0], loc='upper right')

sns.histplot(
    df_oligos_long,
    x='count',
    weights='count',
    hue='oligo species',
    bins=bins,
    alpha=0.3,
    ax=axs[1, 1]
)
axs[1, 1].set_xscale(xscale)
axs[1, 1].set_xlim(-1, None)
axs[1, 1].set_yscale('symlog', linthresh=10)
axs[1, 1].set_ylabel('number of oligo reads')
sns.move_legend(axs[1, 1], loc='upper right')

fig.supxlabel('number of oligo molecules per bead')
fig.suptitle('Distribution of oligo molecules per bead\n(on oligo-containing beads)')
fig.savefig(
    os.path.join(DIR_RESULTS, 'oligo molecules per bead.png'),
    dpi=300,
    bbox_inches='tight'
)

del df_oligos_long
gc.collect()
fig.show()

In [ ]:
fig, axs = plt.subplots(2, 1, sharex=True, constrained_layout=True)
sns.histplot(
    df_oligo_species_per_bead.loc[df_oligo_species_per_bead['total'] > 1]
    .pipe(lambda df: df.assign(**{
        'oligo molecules per bead': df['oligo molecules per bead'].cat.remove_categories('1')
    })),
    x='human',
    stat='proportion',
    multiple='stack',
    common_norm=False,
    hue='oligo molecules per bead',
    palette='viridis',
    ax=axs[0]
)
sns.ecdfplot(
    df_oligo_species_per_bead.loc[df_oligo_species_per_bead['total'] > 1]
    .pipe(lambda df: df.assign(**{
        'oligo molecules per bead': df['oligo molecules per bead'].cat.remove_categories('1')
    })),
    x='human',
    hue='oligo molecules per bead',
    palette='viridis',
    legend=False,
    ax=axs[1]
)
axs[0].set_ylabel('proportion of beads')
axs[1].set_ylabel('proportion of beads')
axs[1].set_xlabel('proportion of oligo molecules on a bead\nfrom human genome (vs. mouse genome)')
axs[0].set_title('Distribution of oligo species on beads with >= 2 oligos')
fig.savefig(
    os.path.join(DIR_RESULTS, 'oligo species on beads.png'),
    dpi=300,
    bbox_inches='tight'
)
fig.show()

## Mixing analysis

Note: only performed using paired-end alignments.

### Beads with only 1 chromatin species

In [ ]:
# index = bead
# columns = chromatin species, chromatin count, human oligo count, mouse oligo count
df_single_chromatin_species_beads = (
    df_chromatin.loc[df_chromatin['alignment_type'] == 'PE', ['bead', 'species']]
    .groupby('bead')
    .filter(lambda g: len(g['species'].unique()) == 1)
    .groupby('bead')
    ['species'].agg(['first', 'size'])
    .rename(columns={'size': 'chromatin count', 'first': 'chromatin species'})
    .merge(df_oligos, how='left', left_index=True, right_index=True)
    .fillna({'human': 0, 'mouse': 0})
    .astype({'human': int, 'mouse': int})
    .rename(columns={'human': 'human oligo count', 'mouse': 'mouse oligo count'})
)

Oligo counts for single-chromatin-species beads

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)
ax = sns.scatterplot(
    (
        df_single_chromatin_species_beads
        .groupby(['chromatin species', 'human oligo count', 'mouse oligo count'], observed=True)
        .size()
        .rename('bead count')
        .reset_index()
        .rename(columns={'chromatin species': 'bead chromatin species'})
        # x-jitter human oligo counts for visualization
        .pipe(lambda df: df.assign(**{
            'human oligo count': df['human oligo count'] + 0.2 * (df['bead chromatin species'] == 'human').astype(float) - 0.1
        }))
    ),
    x='human oligo count',
    y='mouse oligo count',
    hue='bead chromatin species',
    size='bead count',
    sizes=(10, 50),
    ax=ax
)
# ax.set_aspect('equal')
ax.set_title('Oligo counts for single-chromatin-species beads')
ax.xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
ax.figure.savefig(
    os.path.join(DIR_RESULTS, 'single-chromatin-species-beads_mixing.png'),
    bbox_inches='tight',
    dpi=300
)

In [ ]:
ax = sns.scatterplot(
    (
        df_single_chromatin_species_beads
        .groupby(['chromatin species', 'human oligo count', 'mouse oligo count'], observed=True)
        .size()
        .rename('bead count')
        .reset_index()
        .rename(columns={'chromatin species': 'bead chromatin species'})
        # remove beads with no oligos
        .pipe(lambda df: df.loc[(df['human oligo count'] + df['mouse oligo count']) > 0])
        # x-jitter human oligo counts for visualization
        .pipe(lambda df: df.assign(**{
            'human oligo count': df['human oligo count'] + 0.2 * (df['bead chromatin species'] == 'human').astype(float) - 0.1
        }))
    ),
    x='human oligo count',
    y='mouse oligo count',
    hue='bead chromatin species',
    size='bead count',
    sizes=(10, 150)
)
ax.set_xlim((-0.5, 10.5))
ax.set_ylim((-0.5, 10.5))
ax.set_aspect('equal')
ax.set_title('Oligo counts for single-chromatin-species beads')
ax.xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
ax.figure.savefig(
    os.path.join(DIR_RESULTS, 'single-chromatin-species-beads_mixing-zoom.png'),
    bbox_inches='tight',
    dpi=300
)

Distribution of oligos on beads with single chromatin species

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 4), constrained_layout=True)
for i, species in enumerate(SPECIES):
    axs[i].set_title(f'beads with {species}-only chromatin')
    sns.barplot(
        # bead, chromatin count, oligo count, oligo species
        (
            df_single_chromatin_species_beads
            .loc[df_single_chromatin_species_beads['chromatin species'] == species]
            .reset_index()
            .rename(columns={'human oligo count': 'human', 'mouse oligo count': 'mouse'})
            .melt(
                id_vars=['bead', 'chromatin count'],
                value_vars=['human', 'mouse'],
                var_name='oligo species',
                value_name='oligo count'
            )
        ),
        x='chromatin count',
        y='oligo count',
        hue='oligo species',
        errorbar='se',
        ax=axs[i]
    )
fig.savefig(
    os.path.join(DIR_RESULTS, 'single-chromatin-species-beads_oligo-counts.png'),
    bbox_inches='tight',
    dpi=300
)

### Beads with only 1 chromatin species and 1 chromatin target

In [ ]:
df_single_chromatin_species_target_beads = (
    df_chromatin.loc[df_chromatin['alignment_type'] == 'PE', ['bead', 'species', 'target']]
    .groupby('bead')
    .filter(lambda g: (len(g['species'].unique()) == 1) and (len(g['target'].unique()) == 1))
    .rename(columns={'species': 'chromatin species'})
    .groupby(['bead', 'chromatin species', 'target'], observed=True)
    .size()
    .rename('chromatin count')
    .reset_index()
    .merge(df_oligos, how='left', left_on='bead', right_index=True)
    .fillna({'human': 0, 'mouse': 0})
    .astype({'human': int, 'mouse': int})
    .rename(columns={'human': 'human oligo count', 'mouse': 'mouse oligo count'})
)

In [ ]:
df_oligos['human']

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True, constrained_layout=True)
axs[0] = sns.scatterplot(
    (
        df_single_chromatin_species_target_beads
        .loc[df_single_chromatin_species_target_beads['target'] == 'CTCF']
        .groupby(['chromatin species', 'human oligo count', 'mouse oligo count'], observed=True)
        .size()
        .rename('bead count')
        .reset_index()
        .rename(columns={'chromatin species': 'bead chromatin species'})
        # x-jitter human oligo counts for visualization
        .pipe(lambda df: df.assign(**{
            'human oligo count': df['human oligo count'] + 0.2 * (df['bead chromatin species'] == 'human').astype(float) - 0.1
        }))
    ),
    x='human oligo count',
    y='mouse oligo count',
    hue='bead chromatin species',
    size='bead count',
    sizes=(10, 50),
    ax=axs[0]
)
# ax.set_aspect('equal')
axs[0].set_title('Oligo counts for single-chromatin-species CTCF beads')
axs[0].xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))
sns.move_legend(axs[0], "upper left", bbox_to_anchor=(1, 1))

axs[1] = sns.scatterplot(
    (
        df_single_chromatin_species_target_beads
        .loc[df_single_chromatin_species_target_beads['target'] == 'H3K4me3']
        .groupby(['chromatin species', 'human oligo count', 'mouse oligo count'], observed=True)
        .size()
        .rename('bead count')
        .reset_index()
        .rename(columns={'chromatin species': 'bead chromatin species'})
        # x-jitter human oligo counts for visualization
        .pipe(lambda df: df.assign(**{
            'human oligo count': df['human oligo count'] + 0.2 * (df['bead chromatin species'] == 'human').astype(float) - 0.1
        }))
    ),
    x='human oligo count',
    y='mouse oligo count',
    hue='bead chromatin species',
    size='bead count',
    sizes=(10, 50),
    ax=axs[1]
)
axs[1].set_title('Oligo counts for single-chromatin-species H3K4me3 beads')
axs[1].xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))
sns.move_legend(axs[1], "upper left", bbox_to_anchor=(1, 1))

fig.show()
ax.figure.savefig(
    os.path.join(DIR_RESULTS, 'single-chromatin-species_single-target_beads_mixing.png'),
    bbox_inches='tight',
    dpi=300
)

## Motif analysis

1. If I have enough reads for peak calling: makeTagDirectory --> findPeaks --> findMotifsGenome.pl
2. If not enough reads for peak calling: findMotifsGenome.pl

Potential concern: reads aligning to highly conserved regions between mouse and human genomes (which may include conserved CTCF binding sites or promoter regions normally marked by H3K4me3) may have been filtered out by the pipeline. Consequently, motif analysis performed on reads filtered for with high mapping quality to the combined genome is likely extremely conservative.

# Clean

In [ ]:
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(
                          globals().items())), key= lambda x: -x[1])[:20]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

In [ ]:
# %%bash -s {DIR_PROC} {DIR_AUX} {DIR_SCRIPTS}
# DIR_PROC="$1"
# DIR_AUX="$2"
# DIR_SCRIPTS="$3"

# source ~/.bashrc
# conda activate snakemake

# snakemake \
# --snakefile "${DIR_SCRIPTS}/Snakefile" \
# --configfile "${DIR_AUX}/config.yaml" \
# -j 1 \
# clean